In [ ]:
!pip install lazypredict

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer as s
from sklearn.impute import KNNImputer as knn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
data = pd.read_csv(r'train.csv')
data_pred=pd.read_csv(r'test.csv')

In [ ]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
data=data.drop('PassengerId',axis=1)
data=data.drop('Name',axis=1)

In [ ]:
data

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [ ]:
data['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [ ]:
data_pred.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object

In [ ]:
data.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

In [ ]:
def Label_Encoding(data):
        enc=[]
        x=pd.DataFrame(data.dtypes)
        ll=list(x[x[0]=="object"].index)
        for i in ll:
            label_encoder = LabelEncoder()
            label_encoder.fit(data[i])
            data[i]= label_encoder.transform(data[i])
            enc.append(label_encoder)
        return (data,enc)

In [ ]:
le=Label_Encoding(data)

In [ ]:
le

In [ ]:
data.dtypes

HomePlanet        int64
CryoSleep         int64
Destination       int64
Age             float64
VIP               int64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name              int64
Transported        bool
dtype: object

In [ ]:
data=le[0]

In [ ]:
def get_all_Null(data,dtype=""):
        x=data.isna().sum()>0
        l=[]
        for i in  list(x.index):
            thresh=data[i].isna().sum()/len(data)
            if(x[i]==True and (data[i].dtypes==dtype) ):
                print(i,data[i].isna().sum())
                l+=[i]
        return l

In [ ]:
get_all_Null(data_pred,'float64')

[]

In [ ]:
data=data.drop('Cabin',axis=1)

In [ ]:
data['ShoppingMall']=data['ShoppingMall'].fillna(data['ShoppingMall'].mean())

In [ ]:
data['RoomService']=data['RoomService'].fillna(data['RoomService'].mean())

In [ ]:
data['FoodCourt']=data['FoodCourt'].fillna(data['FoodCourt'].mean())

In [ ]:
data['Spa']=data['Spa'].fillna(data['Spa'].mean())

In [ ]:
data['VRDeck']=data['VRDeck'].fillna(data['VRDeck'].mean())

In [ ]:
def knn_impute(data,n_neighbors,col_list):
        imputer=knn(n_neighbors=n_neighbors)
        for i in col_list:
            data[i]=imputer.fit_transform(data[[i]])[0][0]
        return data

In [ ]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
data=knn_impute(data,3,['Age'])

In [ ]:
from sklearn.model_selection import train_test_split
x=d1
y=data.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2024)

In [ ]:
data.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported        bool
dtype: object

In [ ]:
data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False
1,0,0,2,39.0,0,109.0,9.0,25.0,549.0,44.0,True
2,1,0,2,39.0,1,43.0,3576.0,0.0,6715.0,49.0,False
3,1,0,2,39.0,0,0.0,1283.0,371.0,3329.0,193.0,False
4,0,0,2,39.0,0,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,39.0,1,0.0,6819.0,0.0,1643.0,74.0,False
8689,0,1,1,39.0,0,0.0,0.0,0.0,0.0,0.0,False
8690,0,0,2,39.0,0,0.0,0.0,1872.0,1.0,0.0,True
8691,1,0,0,39.0,0,0.0,1049.0,0.0,353.0,3235.0,False


In [ ]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_train,y_train)

LogisticRegression()

In [ ]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(x_train, x_test, y_train, y_test)


100%|██████████| 29/29 [00:13<00:00,  2.14it/s]

[LightGBM] [Info] Number of positive: 3057, number of negative: 3028
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1293
[LightGBM] [Info] Number of data points in the train set: 6085, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502383 -> initscore=0.009532
[LightGBM] [Info] Start training from score 0.009532


In [ ]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LinearSVC                          0.78               0.78     0.78      0.78   
RandomForestClassifier             0.78               0.78     0.78      0.78   
CalibratedClassifierCV             0.78               0.78     0.78      0.78   
LogisticRegression                 0.77               0.77     0.77      0.77   
AdaBoostClassifier                 0.77               0.77     0.77      0.77   
SGDClassifier                      0.77               0.77     0.77      0.77   
ExtraTreesClassifier               0.77               0.77     0.77      0.77   
BaggingClassifier                  0.77               0.77     0.77      0.77   
LGBMClassifier                     0.77               0.77     0.77      0.77   
LabelPropagation                   0.77               0.77     0.77      0.77   
XGBClassifier               

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 300}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap=True,criterion='entropy',n_estimators=300, random_state=0)
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7753067484662577

criterion{“gini”, “entropy”, “log_loss”}

In [ ]:
from sklearn.model_selection import GridSearchCV
param={
    'n_estimators':[i for i in range(100,301,100)],
    'criterion':['gini','entropy','log_loss'],
    'min_samples_split':[i for i in range(2,10)],
    'max_features':['sqrt','log2',None],
    'bootstrap':[True,False],

}

In [ ]:
data

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,False
1,Earth,False,TRAPPIST-1e,39.00,False,109.00,9.00,25.00,549.00,44.00,True
2,Europa,False,TRAPPIST-1e,39.00,True,43.00,3576.00,0.00,6715.00,49.00,False
3,Europa,False,TRAPPIST-1e,39.00,False,0.00,1283.00,371.00,3329.00,193.00,False
4,Earth,False,TRAPPIST-1e,39.00,False,303.00,70.00,151.00,565.00,2.00,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,39.00,True,0.00,6819.00,0.00,1643.00,74.00,False
8689,Earth,True,PSO J318.5-22,39.00,False,0.00,0.00,0.00,0.00,0.00,False
8690,Earth,False,TRAPPIST-1e,39.00,False,0.00,0.00,1872.00,1.00,0.00,True
8691,Europa,False,55 Cancri e,39.00,False,0.00,1049.00,0.00,353.00,3235.00,False


In [ ]:
clf1=RandomForestClassifier()
gr=GridSearchCV(clf1,param_grid=param,cv=5)
gr.fit(x_train,y_train)
gr.best_params_

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 300}

In [ ]:
def StdScale(data):
        for i in data.columns:
            if data[i].dtypes!='object' and i!='SalePrice':
                scale = StandardScaler().fit(data[[i]])

                data[i] = scale.transform(data[[i]])
        return data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
d1=StdScale(data.iloc[:,:-1])

In [ ]:
d1

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.33,-0.74,0.58,0.00,-0.21,-0.34,-0.29,-0.29,-0.28,-0.27
1,-0.83,-0.74,0.58,0.00,-0.21,-0.18,-0.28,-0.25,0.21,-0.23
2,0.33,-0.74,0.58,0.00,2.79,-0.28,1.96,-0.29,5.69,-0.23
3,0.33,-0.74,0.58,0.00,-0.21,-0.34,0.52,0.33,2.68,-0.10
4,-0.83,-0.74,0.58,0.00,-0.21,0.12,-0.24,-0.04,0.23,-0.27
...,...,...,...,...,...,...,...,...,...,...
8688,0.33,-0.74,-1.80,0.00,2.79,-0.34,3.99,-0.29,1.18,-0.20
8689,-0.83,1.12,-0.61,0.00,-0.21,-0.34,-0.29,-0.29,-0.28,-0.27
8690,-0.83,-0.74,0.58,0.00,-0.21,-0.34,-0.29,2.84,-0.28,-0.27
8691,0.33,-0.74,-1.80,0.00,-0.21,-0.34,0.37,-0.29,0.04,2.59


In [ ]:
data_pred

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.00,False,0.00,0.00,0.00,0.00,0.00,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.00,False,0.00,9.00,0.00,2823.00,0.00,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.00,False,0.00,0.00,0.00,0.00,0.00,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.00,False,0.00,6652.00,0.00,181.00,585.00,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.00,False,10.00,0.00,635.00,0.00,0.00,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.00,False,0.00,0.00,0.00,0.00,0.00,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.00,False,0.00,847.00,17.00,10.00,144.00,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.00,0.00,0.00,0.00,0.00,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.00,2680.00,0.00,0.00,523.00,Kitakan Conale


In [ ]:
dd11=list(data.columns)
dd11.remove('Transported')

In [ ]:
data_pred=data_pred[dd11]

In [ ]:
le[-1]

LabelEncoder()

In [ ]:
x=pd.DataFrame(data_pred.dtypes)
ll=list(x[x[0]=="object"].index)
for i in ll:
    data_pred[i]= le[-1].transform(data_pred[i])

ValueError: y contains previously unseen labels: 'Earth'

In [ ]:
data_pred=pd.read_csv(r'test.csv')

In [ ]:

data_pred['ShoppingMall']=data_pred['ShoppingMall'].fillna(data_pred['ShoppingMall'].mean())
data_pred['RoomService']=data_pred['RoomService'].fillna(data_pred['RoomService'].mean())
data_pred['FoodCourt']=data_pred['FoodCourt'].fillna(data_pred['FoodCourt'].mean())
data_pred['Spa']=data_pred['Spa'].fillna(data_pred['Spa'].mean())
data_pred['VRDeck']=data_pred['VRDeck'].fillna(data_pred['VRDeck'].mean())
data_pred=knn_impute(data_pred,3,['Age'])


In [ ]:
le[-1]

[LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()]

In [ ]:
data_pred.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object

In [ ]:
x=pd.DataFrame(data_pred.dtypes)
ll=list(x[x[0]=="object"].index)

In [ ]:
x


,0
PassengerId,object
HomePlanet,object
CryoSleep,object
Cabin,object
Destination,object
Age,float64
VIP,object
RoomService,float64
FoodCourt,float64
ShoppingMall,float64


In [ ]:
label_encoder = le[-1]
x=pd.DataFrame(data_pred.dtypes)
ll=list(x[x[0]=="object"].index)
j=0
for i in ll:
    data_pred[i]= label_encoder[j].transform(data_pred[i])
    j+=1

In [ ]:
data_pred

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,27.0,0,0.0,9.0,0.0,2823.0,0.0
2,1,1,0,27.0,0,0.0,0.0,0.0,0.0,0.0
3,1,0,2,27.0,0,0.0,6652.0,0.0,181.0,585.0
4,0,0,2,27.0,0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0
4273,0,0,2,27.0,0,0.0,847.0,17.0,10.0,144.0
4274,2,1,0,27.0,0,0.0,0.0,0.0,0.0,0.0
4275,1,0,3,27.0,0,0.0,2680.0,0.0,0.0,523.0


In [ ]:
le[-1][3].classes_

array([False, True, nan], dtype=object)

In [ ]:
d=StdScale(data_pred)

In [ ]:
y_pred=clf.predict(d)

In [ ]:
data_pred1=pd.read_csv(r'test.csv')

In [ ]:
data_pred1['PassengerId']

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [ ]:
y_pred=np.ndarray.flatten(y_pred)

In [ ]:
y_pred=pd.Series(y_pred)
ds=pd.concat([pd.Series(data_pred1['PassengerId'].values),y_pred],axis=1)
ds.columns=['PassengerId','Transported']
ds.to_csv('ANN_hyper_param_tuned.csv',index=False)

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [ ]:
model=Sequential()
model.add(Dense(units=100,kernel_initializer='he_uniform',activation='relu',input_dim=10))
model.add(Dense(units=50,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='he_uniform',activation='sigmoid'))
learning_rate= 0.017995567894057134
optimizer=Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
mh_=model.fit(x_train,y_train,validation_split=0.2,batch_size=10,epochs=100)

Epoch 1/100
487/487 [==============================] - 3s 4ms/step - loss: 0.4958 - accuracy: 0.7705 - val_loss: 0.4377 - val_accuracy: 0.7913
Epoch 2/100
487/487 [==============================] - 1s 3ms/step - loss: 0.4585 - accuracy: 0.7851 - val_loss: 0.4636 - val_accuracy: 0.8028
Epoch 3/100
487/487 [==============================] - 1s 3ms/step - loss: 0.4519 - accuracy: 0.7890 - val_loss: 0.4176 - val_accuracy: 0.7929
Epoch 4/100
487/487 [==============================] - 1s 2ms/step - loss: 0.4549 - accuracy: 0.7940 - val_loss: 0.4124 - val_accuracy: 0.8012
Epoch 5/100
487/487 [==============================] - 1s 3ms/step - loss: 0.4482 - accuracy: 0.7925 - val_loss: 0.4306 - val_accuracy: 0.7929
Epoch 6/100
487/487 [==============================] - 1s 2ms/step - loss: 0.4477 - accuracy: 0.7923 - val_loss: 0.4155 - val_accuracy: 0.8020
Epoch 7/100
487/487 [==============================] - 1s 3ms/step - loss: 0.4432 - accuracy: 0.7931 - val_loss: 0.4136 - val_accuracy: 0.8036

In [ ]:
y_pred=model.predict(d)

134/134 [==============================] - 0s 1ms/step


In [ ]:
y_pred=np.round(y_pred).astype(bool)

In [ ]:
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score


In [ ]:
import optuna
def obj(trial):
  units=trial.suggest_categorical('units',[50,100,150])
  activation=trial.suggest_categorical('activation',['relu','sigmoid'])
  learning_rate=trial.suggest_float('learning_rate',1e-4,1e-1,log=True)

  optimizer_name=trial.suggest_categorical('optimizer_name',['adam','sgd'])
  model=Sequential()
  model.add(Dense(units=units,kernel_initializer='he_uniform',activation=activation,input_dim=10))
  model.add(Dense(units=units,kernel_initializer='he_uniform',activation=activation))
  model.add(Dense(units=1,kernel_initializer='he_uniform',activation='sigmoid'))
  if optimizer_name=='adam':
    optimizer=Adam(learning_rate=learning_rate)
  else:
    optimizer=optimizer_name

  model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
  model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=0)
  y_pred=model.predict(x_test)
  y_pred=np.round(y_pred).astype(bool)
  y_pred=np.ndarray.flatten(y_pred)
  return accuracy_score(y_test,y_pred)


In [ ]:
from sklearn.model_selection import train_test_split
x=d1
y=data.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2024)

In [ ]:
ht=optuna.create_study(direction='maximize')
ht.optimize(obj,n_trials=100)
ht.best_params

[I 2024-04-15 12:33:27,989] A new study created in memory with name: no-name-c5075dac-8c5b-4745-b5f2-6a77ee06ad1a


82/82 [==============================] - 0s 2ms/step


[I 2024-04-15 12:33:50,672] Trial 0 finished with value: 0.7733895705521472 and parameters: {'units': 100, 'activation': 'sigmoid', 'learning_rate': 0.0001920645927976415, 'optimizer_name': 'adam'}. Best is trial 0 with value: 0.7733895705521472.


82/82 [==============================] - 0s 1ms/step


[I 2024-04-15 12:34:01,757] Trial 1 finished with value: 0.7776073619631901 and parameters: {'units': 50, 'activation': 'sigmoid', 'learning_rate': 0.001313064153542927, 'optimizer_name': 'adam'}. Best is trial 1 with value: 0.7776073619631901.


...

82/82 [==============================] - 0s 2ms/step


[I 2024-04-15 12:55:41,946] Trial 98 finished with value: 0.7799079754601227 and parameters: {'units': 100, 'activation': 'relu', 'learning_rate': 0.006851629155017264, 'optimizer_name': 'adam'}. Best is trial 89 with value: 0.7825920245398773.


82/82 [==============================] - 0s 1ms/step


[I 2024-04-15 12:55:52,517] Trial 99 finished with value: 0.7745398773006135 and parameters: {'units': 100, 'activation': 'relu', 'learning_rate': 0.0051728331620880825, 'optimizer_name': 'sgd'}. Best is trial 89 with value: 0.7825920245398773.


{'units': 100,
 'activation': 'relu',
 'learning_rate': 0.017995567894057134,
 'optimizer_name': 'adam'}